In [1]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001)
import numpy as np
import pandas as pd
import math
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
from keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Add, Dense, Layer, Activation, concatenate, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow_addons.layers import WeightNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

warnings.filterwarnings("ignore")
%matplotlib inline

# Clear the session
# K.clear_session()

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

class PrintShapeCallback(Callback):
    def __init__(self, model_layer_name):
        super(PrintShapeCallback, self).__init__()
        self.model_layer_name = model_layer_name

    def on_epoch_end(self, epoch, logs=None):
        # Get the output of the layer with the specified name
        layer_output = self.model.get_layer(self.model_layer_name).output
        print(f"After epoch {epoch+1}, shape of x (from layer {self.model_layer_name}): {layer_output.shape}")

print_shape_callback = PrintShapeCallback(model_layer_name='dense_1')  # Assuming 'dense_1' is the name of the Dense layer after h2(x)

class StopAtThresholdCallback(Callback):
    def __init__(self, threshold):
        super(StopAtThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_loss = logs.get('val_loss')
        if val_loss is not None and val_loss < self.threshold:
            print(f"\nStopping training as validation loss {val_loss} is below the threshold {self.threshold}")
            self.model.stop_training = True

# callback = StopAtThresholdCallback(threshold=1e-03)
callback = StopAtThresholdCallback(threshold=1.2962e-07)

class H1Layer(Layer):
    def __init__(self, **kwargs):
        super(H1Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.b = self.add_weight(shape=(input_shape[-1],),
                                initializer=RandomNormal(mean=0.0,stddev=0.03),
                                trainable=True)
        super(H1Layer, self).build(input_shape)

    def call(self, x):
        return (self.b * (2 * x))/ (2**(1/2) * np.pi**(1/4))
        #return (2 * x) 


class H2Layer(Layer):
    def __init__(self, **kwargs):
        super(H2Layer, self).__init__(**kwargs)

    def call(self, x, h1):
        return (((2*x*(h1)))-2)/(2*(np.pi**(1/4))*np.sqrt(math.factorial(2)))
    
class H3Layer(Layer):
    def __init__(self, **kwargs):
        super(H3Layer, self).__init__(**kwargs)
        
    def call(self, x, h1, h2):
        return (((2 * x * (h2)))-(4 * h1)) / (2**(3/2)*(np.pi**(1/4))*np.sqrt(math.factorial(3)))

class H4Layer(Layer):
    def __init__(self, **kwargs):
        super(H4Layer, self).__init__(**kwargs)

    def call(self, x, h2, h3):
        return (((2*x*(h3)))-(6*h2)) / (2**2 *(np.pi**(1/4))*np.sqrt(math.factorial(4)))

class H5Layer(Layer):
    def __init__(self, **kwargs):
        super(H5Layer,self).__init__(**kwargs)

    def call(self,x, h3, h4):
        return (((2 * x * h4)) - (8 * h3)) / (2**(5/2) * (np.pi**(1/4)) * np.sqrt(math.factorial(5)))

#FROM THIS ON NO MODIFICATION ON WEIGHTS
class H6Layer(Layer):
    def __init__(self, **kwargs):
        super(H6Layer,self).__init__(**kwargs)
        
    def call(self,x, h4, h5):
        return (2*x*(h5))-(10*h4)

class TensorDecompositionLayer(Layer):
    def __init__(self, rank, **kwargs):
        self.rank = rank
        super(TensorDecompositionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.factors_a = self.add_weight(shape=(input_shape[-1], self.rank),
                                         initializer=RandomNormal(mean=0.0,stddev=0.05),
                                         trainable=True)
        self.factors_b = self.add_weight(shape=(self.rank, input_shape[-1]),
                                        initializer=RandomNormal(mean=0.0,stddev=0.05),
                                        trainable=True)
        super(TensorDecompositionLayer, self).build(input_shape)

    def call(self, x):
        return tf.matmul(tf.matmul(x, self.factors_a), self.factors_b)

 
class Relu_With_Weight(Layer):
    def __init__(self, **kwargs):
        super(Relu_With_Weight, self).__init__(**kwargs)

    def build(self, input_shape):
        self.b = self.add_weight(shape=(input_shape[-1],),
                                initializer=RandomNormal(),
                                trainable=True)
        super(Relu_With_Weight, self).build(input_shape)

    def call(self, x):
        return K.tanh(x * self.b)

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the images to a 0 to 1 range
X_train, X_val = x_train / 255.0, x_test / 255.0

# Convert class vectors to binary class matrices (one-hot encoding)
y_train, y_val = to_categorical(y_train, 10), to_categorical(y_test, 10)

# Printing the shape of the dataset to confirm
print("Training data shape:", x_train.shape, y_train.shape)
print("Test data shape:", x_test.shape, y_test.shape)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(X_train)

def build_model(input_shape, num_classes, filters):
    rank = 5
    input_layer = Input(shape=input_shape)
    x = input_layer

    h1 = H1Layer()
    h2 = H2Layer()
    h3 = H3Layer()
    h4 = H4Layer()
    h5 = H5Layer()
    x = BatchNormalization()(x)

    # x = WeightNormalization(Conv2D(filters=filters, kernel_size=(7,7), strides=2, padding='same',kernel_initializer="he_normal"))(x)
    x = Conv2D(filters=filters, kernel_size=(7,7), strides=2, padding='same',kernel_initializer="he_normal")(x)

    x = x_h1 = h1(x)
    # x = WeightNormalization(Conv2D(filters=filters, kernel_size=(5, 5), padding='same',kernel_initializer="he_normal"))(x)
    x = Conv2D(filters=filters, kernel_size=(5, 5), padding='same',kernel_initializer="he_normal")(x)
    #x = TensorDecompositionLayer(rank)(x)
    x = x_h2 = h2(x,x_h1)
    # x = WeightNormalization(Conv2D(filters=filters, kernel_size=(3, 3), padding='same',kernel_initializer="he_normal"))(x)
    x = Conv2D(filters=filters, kernel_size=(3, 3), padding='same',kernel_initializer="he_normal")(x)
    x = x_h3 = h3(x, x_h1, x_h2)
    x = Conv2D(filters=filters, kernel_size=(3, 3), padding='same',kernel_initializer="he_normal")(x)
    x = x_h4 = h4(x, x_h2, x_h3)
    x = Conv2D(filters=filters, kernel_size=(3, 3), padding='same',kernel_initializer="he_normal")(x)
    x = x_h5 = h5(x, x_h3, x_h4)
    x = Conv2D(filters=filters, kernel_size=(3, 3), padding='same',kernel_initializer="he_normal")(x)
    
    x = GlobalAveragePooling2D()(x)
    # x = Dropout(0.5)(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu', kernel_initializer="he_normal")(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

input_shape = X_train.shape[1:]
num_classes = 10
filters = 64

model = build_model(input_shape, num_classes, filters)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001)

history = model.fit(datagen.flow(X_train, y_train, batch_size=128),
                    validation_data=(X_val, y_val),
                    epochs=50,
                    callbacks=[])

# Evaluate the model
score = model.evaluate(X_val, y_val)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Plot the accuracy and loss
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

2024-06-05 14:55:53.614789: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 14:55:53.704218: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-05 14:55:53.725799: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-05 14:55:54.144070: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Training data shape: (50000, 32, 32, 3) (50000, 10)
Test data shape: (10000, 32, 32, 3) (10000, 1)


2024-06-05 14:55:55.931545: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-05 14:55:55.945972: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-05 14:55:55.946098: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-05 14:55:55.946504: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

Epoch 1/50


2024-06-05 14:55:56.984217: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2024-06-05 14:55:57.073114: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


391/391 [==============================] - 15s 37ms/step - loss: 2.0149 - accuracy: 0.2299 - val_loss: 1.7700 - val_accuracy: 0.3358
Epoch 2/50
391/391 [==============================] - 14s 36ms/step - loss: 1.7686 - accuracy: 0.3255 - val_loss: 1.6723 - val_accuracy: 0.3693
Epoch 3/50
391/391 [==============================] - 14s 36ms/step - loss: 1.7427 - accuracy: 0.3538 - val_loss: 1.6327 - val_accuracy: 0.3937
Epoch 4/50
391/391 [==============================] - 14s 37ms/step - loss: 1.6577 - accuracy: 0.3765 - val_loss: 1.5873 - val_accuracy: 0.4061
Epoch 5/50
391/391 [==============================] - 15s 38ms/step - loss: 1.6151 - accuracy: 0.4001 - val_loss: 1.5617 - val_accuracy: 0.4261
Epoch 6/50
391/391 [==============================] - 14s 37ms/step - loss: 1.5611 - accuracy: 0.4226 - val_loss: 1.4669 - val_accuracy: 0.4606
Epoch 7/50
391/391 [==============================] - 15s 38ms/step - loss: 1.5168 - accuracy: 0.4443 - val_loss: 1.4388 - val_accuracy: 0.4900
Epo

In [9]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 32, 32, 3)   12          ['input_4[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 conv2d_12 (Conv2D)             (None, 16, 16, 32)   4736        ['batch_normalization_3[0][0]']  
                                                                                                  
 h1_layer_3 (H1Layer)           (None, 16, 16, 32)   32          ['conv2d_12[0][0]']        